# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [4]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 1 and isinstance(args[0], list):
            # Matrix initialization with a list of lists
            data = args[0]
            if not all(isinstance(row, list) for row in data):
                raise ValueError("Each row must be a list.")
            if len(set(len(row) for row in data)) > 1:
                raise ValueError("All rows must have the same number of columns.")
            self.data = data
            self.n = len(data)        # Number of rows
            self.m = len(data[0])     # Number of columns
        elif len(args) == 2 and isinstance(args[0], int) and isinstance(args[1], int):
            # Matrix initialization with n and m
            n, m = args
            self.data = [[0.0 for _ in range(m)] for _ in range(n)]
            self.n = n
            self.m = m
        else:
            raise ValueError("Invalid initialization parameters.")

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.data[i][j]
        elif isinstance(index, int):
            return self.data[index]
        else:
            raise IndexError("Invalid index type")

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i, j = index
            self.data[i][j] = value
        elif isinstance(index, int):
            self.data[index] = value
        else:
            raise IndexError("Invalid index type")

    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        return Matrix([[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.data

    def block(self, n0, n1, m0, m1):
        return Matrix([row[m0:m1] for row in self.data[n0:n1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix shapes must match for addition.")
        return Matrix([[self.data[i][j] + N[i, j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix shapes must match for subtraction.")
        return Matrix([[self.data[i][j] - N[i, j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Matrix multiplication is not possible: columns of A must match rows of B.")
        result = [[sum(self.data[i][k] * N[k, j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)]
        return Matrix(result)

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix shapes must match for element-wise multiplication.")
        return Matrix([[self.data[i][j] * N[i, j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        return self.data == N.to_list()

    def __eq__(self, other):
        return self.equals(other)

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        elif isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported multiplication.")

    def __rmul__(self, other):
        return self.scalarmul(other)

    def __str__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.data])


# Standalone functions to create special matrices
def constant(n, m, c):
    return Matrix([[c for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return Matrix([[0.0 for _ in range(m)] for _ in range(n)])

def ones(n, m):
    return Matrix([[1.0 for _ in range(m)] for _ in range(n)])

def eye(n):
    return Matrix([[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])


# Testing Matrix class and operations
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Add
C = A + B
print("\nA + B:")
print(C)

# Subtract
D = A - B
print("\nA - B:")
print(D)

# Multiply
E = A * B
print("\nA * B:")
print(E)

# Scalar multiplication
F = A * 2
print("\nA * 2:")
print(F)

# Transpose
G = A.transpose()
print("\nTranspose of A:")
print(G)

# Shape
print("\nShape of A:", A.shape())

# Block
H = A.block(0, 2, 0, 1)
print("\nBlock of A (first row):")
print(H)

# Identity matrix
I = eye(3)
print("\nIdentity matrix of size 3:")
print(I)

# Constant matrix
J = constant(2, 3, 5)
print("\nConstant 2x3 matrix with value 5:")
print(J)



Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

A + B:
6 8
10 12

A - B:
-4 -4
-4 -4

A * B:
19 22
43 50

A * 2:
2 4
6 8

Transpose of A:
1 3
2 4

Shape of A: (2, 2)

Block of A (first row):
1
3

Identity matrix of size 3:
1.0 0.0 0.0
0.0 1.0 0.0
0.0 0.0 1.0

Constant 2x3 matrix with value 5:
5 5 5
5 5 5
